In [1]:
import os
os.environ['http_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128' 
os.environ['https_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128'
from pathlib import Path

In [2]:
%pwd

'/Net/Groups/BGI/scratch/ppandey/Side_Quest/Text_Summarization_HF/experiments'

In [3]:
os.chdir('../')
%pwd

'/Net/Groups/BGI/scratch/ppandey/Side_Quest/Text_Summarization_HF'

In [4]:
# ENTITY FILE! (copy-paste during modular coding)
# We need to define entity which is used to set the return type of a func. This is done using `dataclasses`
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig: # not normal python class
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
# For configuration.py
from txtsummarizer.constants import *
from txtsummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILEPATH,
        params_filepath = PARAMS_FILEPATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create artifacts folder
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Create directory
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
# COMPONENTS!
import urllib.request as rqst
import zipfile

from txtsummarizer.logging import logger
from txtsummarizer.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(
        self, 
        config: DataIngestionConfig
    ):
        self.config = config

    def download_data_from_url(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = rqst.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! Information: \n{headers}')
        else:
            logger.info(f'{filename} already exists! Size: {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
# Create a pipeline using try-exception block   
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data_from_url()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-28 04:38:29,296: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-04-28 04:38:29,298: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-28 04:38:29,299: INFO: common: created directory at: artifacts]
[2024-04-28 04:38:29,299: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-28 04:38:29,562: INFO: 1286912145: artifacts/data_ingestion/data.zip downloaded! Information: 
Connection: close
Content-Length: 4237248
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "90ded68300fda44cfc36c626d0923da47b28fd0a070dcf2c466e850a89f95ef7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EA90:0E89:12BB6F2:13D557B:662DB673
Accept-Ranges: bytes
Date: Sun, 28 Apr 2024 02:38:29 GMT
Via: 1.1 varnish
X-Served-By: cache-fra-eddf8230049-FRA
X-Cache: 